In [15]:
from importlib import reload
import sys
sys.path.append('..')

import cartopy.crs as ccrs
import cartopy.feature as cfeature 
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import seaborn as sb
import xarray as xr

from src.gogoesgone import processing as pr
from src.gogoesgone import zarr_access as za
reload(pr)
reload(za)

<module 'src.gogoesgone.zarr_access' from '/home/robert/Coding/gogoesgone/notebooks/../src/gogoesgone/zarr_access.py'>

# Performance

## Option-1 

On a local Macbook Pro M1, for around a network speed fluctuating between 30-80 MB/s, getting a mapper for a whole day worth of files (for C13 of ABI-L2-CMPIPF) took around 11:41 minutes. Also, processing (e.g. subsampling (22.7s) and plotting (8.5s)) takes longer with a mapper including so many files. 

## Option-2

__N.B. Numbers in parantheses show values for different instances of running the code__

For roughly the same parameters as above, an hour's worth of files took around (35.6,24.3) seconds. Extrapolating that to 24 hours, brings us to a total time of 14 minutes, which suggests that a whole day's mapper might be better than going hourly. However, processing (subsampling (7.5s,8.1s) and plotting (0.5s,0.6s)) is much quicker. 

## Conclusion?

- Subsampling would usually only be done once, so it doesn't make a big difference here.
- But if your goal is to make plots every 10 minutes for 24 hours... 
  - **Option-1** would take 11:41 + (8.5x24x6/60 = 20.4) ~ **30 minutes**, whereas
  - **Option-2** would take (24.3x24/60 = 9.7,35.6x24/60 = 14.24) + (0.5x24x6/60 = 1.2,0.5x24x6/60 = 1.44) ~ **15.44, 10.9 minutes**

Do what you want with that bit of super-rigorous performance testing! /s

In [ ]:
hour = 17
year = 2020
dayofyear = 24
channel = 13
product = 'ABI-L2-CMIPF'  #'ABI-L1b-RadF' 
satellite = 'goes16' #'goes17'

gs = za.generate_globsearch_string(year, dayofyear, hour, channel, product, satellite)
flist = za.generate_url_list(gs)
m = za.get_mapper_from_mzz(flist)

In [27]:
time = "20200124 16:07:05"
format = "%Y%m%d %H:%M:%S"

channel = 13
product = 'ABI-L2-CMIPF'  #'ABI-L1b-RadF' 
satellite = 'goes16' #'goes17'
interval = 120 # minutes

flist= za.nearest_time_url(time,format, interval, channel, product, satellite)
m = za.get_mapper_from_mzz(flist)

s3://noaa-goes16/ABI-L2-CMIPF/2020/024/16/OR_ABI-L2-CMIPF-M6C13_G16_s20200241610164_e20200241619484_c20200241619577.nc


In [ ]:
# Cloud top heights product for specific time
time = "20200124 16:07:05"
format = "%Y%m%d %H:%M:%S"

channel = None
product = 'ABI-L2-ACHAF'  # cloud top heights 
satellite = 'goes16' #'goes17'
interval = 120

flist= za.nearest_time_url(time,format, interval, channel, product, satellite)
m = za.get_mapper_from_mzz(flist)

In [ ]:
# Cloud top heights product for full day
year = 2020
dayofyear = 24
product = 'ABI-L2-ACHAF'  # cloud top heights
satellite = 'goes16' #'goes17'

gs = za.generate_globsearch_string(year, dayofyear, product=product, satellite=satellite)
flist = za.generate_url_list(gs)
m = za.get_mapper_from_mzz(flist)

In [ ]:
img = pr.Image(m)
extent = (-62,-48,10,20)

subset = img.subset_region_from_latlon_extents(extent, unit="degree")

In [ ]:
sb.set_context('paper')
fig = plt.figure(figsize=(8,5))

ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

im = subset.CMI.isel(t=0).plot(ax=ax,x='lon',y='lat',cmap='cubehelix_r',add_colorbar=False,vmin=280,vmax=300)
subset.CMI.isel(t=0).plot.contour(ax=ax,x='lon',y='lat',cmap='Reds',add_colorbar=False,levels=[290,],linewidths=.5)          
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.coastlines(resolution='10m', color='black', linewidth=.5)

#Add title
plt.title(f"{subset.attrs['time_coverage_start']} to {subset.attrs['time_coverage_end']}")
plt.colorbar(im,orientation='horizontal',label='Channel-13 Brightness Temperature / K')

# # # Draw grid.
gl = ax.gridlines(ccrs.PlateCarree(), linewidth = 1, color = 'w', alpha = 0.5, linestyle=':', draw_labels = True)
gl.top_labels = False
gl.right_labels = False
gl.xlines = True
gl.ylines = True
gl.xlocator = mticker.FixedLocator(np.arange(-180, 180, 2.5))
gl.ylocator = mticker.FixedLocator(np.arange(-90, 90, 2.5))
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'color': 'grey'}
gl.ylabel_style = {'color': 'grey'}